This is a demo experiment for **training**, when running an experiment, load model configuration and dataset first.

Then pass it into the train_eval() function to get the best model, which you can subsequently use to perform inferencing.

In [1]:
import sys
sys.path.append('../')

from tts.data import synthetic_tumor_data, TTSDataset
from tts.config import Config
from train_eval.training import training

In [2]:
# run_seed = np.arange(10) for multiple runs
run_seeds = [0]

for seed in run_seeds:
    # load config
    # NOTE: remember to load best hyperparams found after search!!
    config = Config(n_features=4, n_basis=5, T=1, seed=seed)
    # side note: batch_size seems slightly large here?
    # I would recommend much smaller e.g. 128 or make it a tuneable hyperparameter 
    # (which I did in demo_tuning_experiment.ipynb)
    config.training.batch_size = 3000
    
    # load dataset
    X, ts, ys = synthetic_tumor_data(2000,20,1.0,0.0,seed=seed,equation='wilkerson')
    dataset = TTSDataset(config, (X, ts, ys))

    litmodel = training(seed, config, dataset)
    
    # add inferencing/evaluation code... whatever you may wish to do with a trained model

/home/tl522/miniconda/envs/tabular-dl/lib/python3.8/site-packages/pytorch_lightning/utilities/seed.py:47: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
Global seed set to 0
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/tl522/miniconda/envs/tabular-dl/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: logs/run_0/tts
/home/tl522/miniconda/envs/tabular-dl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have

In [8]:
import numpy as np
from tts.visualize import simple_interactive_plot
from tts.data import  _get_tumor_feature_ranges

In [9]:
def trajectory(t, **x):
    feature_names = ["age","weight","initial_tumor_volume","dosage"]
    features = np.array([x[feature_name] for feature_name in feature_names])
    return litmodel.model.forecast_trajectory(features,t)


simple_interactive_plot(trajectory, 1, (0,2), _get_tumor_feature_ranges("age","weight","initial_tumor_volume","dosage"), n_points=100)

interactive(children=(FloatSlider(value=20.0, description='age', max=80.0, min=20.0, step=0.01), FloatSlider(v…

In [10]:
# Ground truth
from tts.visualize import simple_interactive_plot
from tts.data import _tumor_volume_2,  _get_tumor_feature_ranges
simple_interactive_plot(_tumor_volume_2, 1, (0,2), _get_tumor_feature_ranges("age","weight","initial_tumor_volume","dosage"), n_points=1000)

interactive(children=(FloatSlider(value=20.0, description='age', max=80.0, min=20.0, step=0.01), FloatSlider(v…